

# 911 WRPS Calls Data Final Project_Konsur


A description of the main 911 Waterloo Regional Police Service (WRPS) data sources
http://www.wrps.on.ca/sites/default/files/WRPSOccurrenceData_Year_2014_October_15.csv

The data contains the following fields:

* Occurrence Number
* Geographic Location [long, lat]
* Nearest Intersection Location
* Patrol Division
* Patrol Zone
* Municipality
* Reported [Date and Time]
* Initial Call Type (REMOVED)
* Initial Call Type Description
* Final Call Type
* Final Call Type Description
* Initial Priority (Removed)
* Final Priority [1-9]
* Disposition (Removed)
* Dispatch [Date and Time]
* Arrival [Date and Time]
* Cleared [Date and Time]
* Call Dispatch Delay = Dispath - Reported
* Call Travel Time = Arrival - Dispatch
* Call On-Scene Time = Cleared - Arrived
* Call Response Time = Arrival - Reported
* Call Service Time = Cleared - Dispatch
* Total Unit Service Time = ?????


## Data and Setup

____
** Import numpy and pandas **

In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

AttributeError: module 'numpy' has no attribute 'core'

** Import visualization libraries and set %matplotlib inline. **

In [6]:
from multiprocessing import Process, freeze_support
from pylab import rcParams
from scipy import stats
from sklearn import tree
from sklearn import datasets, linear_model, preprocessing
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

AttributeError: module 'numpy' has no attribute 'core'

** Read in the csv file as a dataframe called df **

In [ ]:
path='C:\\Users\\cck023.DS\\OneDrive\\MSCA\\data\\WRPSOccurrenceData_Year_2015_November3_2016.csv'
path


In [ ]:
df=pd.read_csv(path)
df.head(3)

** Check the info() of the df **

In [ ]:
df.info()

** Check the head of df **

In [ ]:
df1=df.dropna()


In [ ]:
df1.shape

In [ ]:
df1.reset_index(drop=True)
df1.head(5)

In [ ]:
df1.shape

In [ ]:
df1 = df1.drop('Disposition', 1)
df1 = df1.drop('Initial Priority', 1)

In [ ]:
df1 = df1.drop('Initial Call Type', 1)


In [ ]:
df1 = df1.drop('Initial Call Type Description',1)

In [ ]:
df1.shape

In [ ]:
df1.head(5)

In [ ]:
df1.columns = [c.replace(' ', '_') for c in df1.columns]

In [ ]:
df1.info()

In [ ]:
col_name='Final Call Type Description'
col_name

In [ ]:
df1['Reason'] = df1['Final_Call_Type_Description'].apply(lambda Final_Call_Type_Description: Final_Call_Type_Description.split('-')[1])

Y:  Recoding target variable: If 'Final Priority' >= 7, then 1, else 0.

In [ ]:
df1.shape

In [ ]:
df1.columns

In [ ]:

Y = df1.Final_Priority.values
Y

In [ ]:
df1 = df1.drop('Final_Priority', axis=1)
X = df1.as_matrix()
X_cols = df1.columns[0:11]
Y = np.asarray([1 if i>=7 else 0 for i in Y])

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
X[:,12:17]

In [ ]:
X2=X[:,12:17]

In [ ]:
df1.head(5)

In [ ]:
#Creating train and test set.
np.random.seed(2)
X_train, X_test, Y_train, Y_test = train_test_split(X2, Y, train_size=0.7)

 I'll standardize the features so that they are centered around 0 with a standard deviation of 1.

In [ ]:
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train = std_scale.transform(X_train)
X_test = std_scale.transform(X_test)

Quick test of multiple models with cross validation

In [ ]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()
clf4 = ExtraTreesClassifier()


In [ ]:
for clf, label in zip([clf1, clf2, clf3, clf4],
 ['Logistic Regression', 'Random Forest', 'Naive Bayes',
 'Extra Trees', 'SVM', 'Gradient Boosting', 'Decision Tree']):
 scores = cross_validation.cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
 print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
clf6 = GradientBoostingClassifier()
clf7 = tree.DecisionTreeClassifier()


In [ ]:
for clf, label in zip([clf6, clf7],
 ['Gradient Boosting', 'Decision Tree']):
 scores = cross_validation.cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
 print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
#clf5 = SVC()

In [ ]:
#for clf, label in zip([clf5],
# ['SVM']):
# scores = cross_validation.cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
# print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

SVM is causing infinite loop 

Gradient Boosting, Random Forest and Extra trees performs the best, while naive bayes performs the worst. I will drop Logistic Regression and Naive Bayes and will analyze the following models: Decision Tree, Random Forest, Gradient Boosting, SVM, and Extra Trees. Decision Tree

In [ ]:
dt = tree.DecisionTreeClassifier(criterion="gini", min_samples_split=10)
dt.fit(X_train, Y_train)
dt_test_preds = dt.predict_proba(X_test)[:, 1]


In [ ]:
dt_preds = dt.predict(X_test)
print (pd.crosstab(index=Y_test, columns=dt_preds, rownames=["Actual"], colnames=["Pred"]))

In [ ]:
dt_roc = roc_auc_score(Y_test, dt_test_preds)
dt_roc

In [ ]:
fpr_decision_tree, tpr_decision_tree, thresholds_decision_tree = roc_curve(Y_test, dt_test_preds)

In [ ]:
fpr_rand = tpr_rand = np.linspace(0, 1, 10)
rcParams['figure.figsize'] = 10, 10
plt.plot(fpr_decision_tree, tpr_decision_tree, label='Decision Tree (area = %0.2f)' % dt_roc)
plt.plot(fpr_rand, tpr_rand, linestyle='--', label='Random Baseline')
plt.legend(loc='lower right', prop={'size':18})
plt.show()

Random Forest Random Forest Classifier with default parameters

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
rf_test_preds = rf.predict_proba(X_test)[:, 1]

In [ ]:
rf_preds = rf.predict(X_test)
print (pd.crosstab(index=Y_test, columns=rf_preds, rownames=["Actual"], colnames=["Pred"]))

In [ ]:
rf_roc = roc_auc_score(Y_test, rf_test_preds)
roc_auc_score(Y_test, rf_test_preds)

In [ ]:
fpr_rf, tpr_rf, thresholds_rf = roc_curve(Y_test, rf_test_preds)

In [ ]:
plt.plot(fpr_decision_tree, tpr_decision_tree, label='Decision Tree (area = %0.2f)' % dt_roc)
plt.plot(fpr_rf, tpr_rf, label='Random Forest (area = %0.2f)' % rf_roc)
plt.plot(fpr_rand, tpr_rand, linestyle='--', label='Random Baseline')
plt.legend(loc='lower right',prop={'size':18})
plt.show()

*Random Forest Classifier feature importance

In [ ]:
rf_importances = rf.feature_importances_
sorted_indices = np.argsort(rf_importances)
y_pos = range(len(rf_importances))
plt.barh(y_pos, rf_importances[sorted_indices], align='center')
plt.yticks(y_pos, X_cols[sorted_indices])
plt.show()

In [46]:
scores = []
for val in range(1,40):
 rf = RandomForestClassifier(n_estimators = val)
 validated = cross_val_score(rf, X_train, Y_train, cv=10)
 scores.append(validated)

In [ ]:
sns.boxplot(scores)
plt.xlabel('Number of Trees')
plt.ylabel('Classification Scores')
plt.title('Classification Score for Number of Trees')
plt.show()

## Basic Questions

** What are the top 5 zipcodes for 911 calls? **

In [ ]:
df1['Reason'].value_counts()

** What are the top 5 townships (twp) for 911 calls? **

In [ ]:
df1['Patrol Zone'].value_counts().head(5)

In [ ]:
df1['Patrol Division'].value_counts().head(5)

** Take a look at the 'title' column, how many unique title codes are there? **

In [ ]:
df1['Patrol Division'].nunique()

In [ ]:
df1['Municipality'].nunique()

In [ ]:
df1['Final Priority'].nunique()

## Creating new features

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.** 

**For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS. **

** What is the most common Reason for a 911 call based off of this new column? **

** Now use seaborn to create a countplot of 911 calls by Reason. **

In [ ]:
sns.countplot(x='Reason',data=df1,palette='viridis')

In [ ]:
sns.countplot(x='Patrol Zone',data=df1,palette='viridis')

In [ ]:
sns.countplot(x='Patrol Division',data=df1,palette='viridis')

####  WORKS UNTIL HERE  **********************************************

___
** Now let us begin to focus on time information. What is the data type of the objects in the timeStamp column? **

In [ ]:
type(df['timeStamp'].iloc[0])

** You should have seen that these timestamps are still strings. Use [pd.to_datetime](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) to convert the column from strings to DateTime objects. **

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

** You can now grab specific attributes from a Datetime object by calling them. For example:**

    time = df['timeStamp'].iloc[0]
    time.hour

**You can use Jupyter's tab method to explore the various attributes you can call. Now that the timestamp column are actually DateTime objects, use .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column, reference the solutions if you get stuck on this step.**

In [ ]:
time = df['timeStamp'].iloc[0]
time.hour

In [ ]:
df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)
df['Month'] = df['timeStamp'].apply(lambda time: time.month)
df['Day of Week'] = df['timeStamp'].apply(lambda time: time.dayofweek)

In [ ]:
df.head(5)

** Notice how the Day of Week is an integer 0-6. Use the .map() with this dictionary to map the actual string names to the day of the week: **

    dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
dmap

In [ ]:
df['Day of Week'] = df['Day of Week'].map(dmap)

In [ ]:
#smap={'Fire':1,'EMS':0, 'Traffic':2} 
#smap

In [ ]:
#df['Reason'] = df['Reason'].map(smap)
#df.head(100)

In [ ]:
df.head(100)

** Now use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column. **

In [ ]:
sns.countplot(x='Day of Week',data=df,hue='Reason',palette='viridis')

# To relocate the legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

** Now do the same for Month:**

In [ ]:
sns.countplot(x='Month',data=df,hue='Reason',palette='viridis')

# To relocate the legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

** Did you notice something strange about the Plot? **

In [ ]:
# It is missing some months! 9,10, and 11 are not there.

** You should have noticed it was missing some Months, let's see if we can maybe fill in this information by plotting the information in another way, possibly a simple line plot that fills in the missing months, in order to do this, we'll need to do some work with pandas...**

** Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame. **

In [ ]:
byMonth = df.groupby('Month').count()
byMonth.head(15)

** Now create a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
# Could be any column
byMonth['twp'].plot()

** Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column. **

In [ ]:
sns.lmplot(x='Month',y='twp',data=byMonth.reset_index())

**Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method. ** 

In [ ]:
df['Date']=df['timeStamp'].apply(lambda t: t.date())

** Now groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [ ]:
df.groupby('Date').count()['twp'].plot()
plt.tight_layout()

** Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [ ]:
df[df['Reason']=='Traffic'].groupby('Date').count()['twp'].plot()
plt.title('Traffic')
plt.tight_layout()

In [ ]:
df[df['Reason']=='Fire'].groupby('Date').count()['twp'].plot()
plt.title('Fire')
plt.tight_layout()

In [ ]:
df[df['Reason']=='EMS'].groupby('Date').count()['twp'].plot()
plt.title('EMS')
plt.tight_layout()

____
** Now let's move on to creating  heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. There are lots of ways to do this, but I would recommend trying to combine groupby with an [unstack](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) method. Reference the solutions if you get stuck on this!**

In [ ]:
dayHour = df.groupby(by=['Day of Week','Hour']).count()['Reason'].unstack()
dayHour.head()

** Now create a HeatMap using this new DataFrame. **

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(dayHour,cmap='viridis')

** Now create a clustermap using this DataFrame. **

In [ ]:
sns.clustermap(dayHour,cmap='viridis')

** Now repeat these same plots and operations, for a DataFrame that shows the Month as the column. **

In [ ]:
dayMonth = df.groupby(by=['Day of Week','Month']).count()['Reason'].unstack()
dayMonth.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(dayMonth,cmap='viridis')

In [ ]:
sns.clustermap(dayMonth,cmap='viridis')

**Continue exploring the Data however you see fit!**


In [ ]:
df.head(5)